In [1]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from pathlib import Path

from PIL import Image
from PIL import ImageOps
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="2", # specify GPU number
        allow_growth=True
    )
)
set_session(tf.Session(config=config))

In [3]:
category = 0
x = []
y = []

def prepare_dataset(file):    
    try:
        im = Image.open(file)
        arr = np.array(im)
        x.append(arr)
        y.append(category)        
    except:
        print("cannot add to dataset:", file)

# Loop all data and create dataset
for dir in Path('./gear_images_128').iterdir():
    for file in dir.iterdir():
        prepare_dataset(file)
    print(dir, ':', category)
    category += 1

x = np.asarray(x)
y = np.asarray(y)

# Split data into 70% train and 30% test. 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.167, random_state=10)

gear_images_128\axes : 0
gear_images_128\boots : 1
gear_images_128\carabiners : 2
gear_images_128\crampons : 3
gear_images_128\gloves : 4
gear_images_128\hardshell_jackets : 5
gear_images_128\harnesses : 6
gear_images_128\helmets : 7
gear_images_128\insulated_jackets : 8
gear_images_128\pulleys : 9
gear_images_128\rope : 10
gear_images_128\tents : 11


In [4]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(1485, 128, 128, 3)
(530, 128, 128, 3)
(107, 128, 128, 3)


In [5]:
y.shape

(2122,)

In [6]:
y

array([ 0,  0,  0, ..., 11, 11, 11])

In [4]:
nb_classes = 12
img_rows, img_cols = 128, 128
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

top_model = Sequential()
top_model = vgg16_model.output
top_model = Flatten()(top_model)
top_model = Dense(2048, activation='relu')(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(1024, activation='relu')(top_model)
predictions = Dense(nb_classes, activation='softmax')(top_model)

model = Model(inputs=vgg16_model.input, outputs=predictions)

for layer in model.layers[:15]:
    layer.trainable = False
model.compile(loss='sparse_categorical_crossentropy', optimizer=SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [5]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
nb_epoch = 10
early_stopping = EarlyStopping(monitor='val_loss', patience=10 , verbose=1)
checkpointer = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True)
model.fit(x=x_test, y=y_test, batch_size=128, epochs=nb_epoch, verbose=1, validation_data=(x_val, y_val), callbacks=[early_stopping, checkpointer])

Train on 530 samples, validate on 107 samples
Epoch 1/10
530/530 [==============================] - 90s 171ms/step - loss: 9.8627 - acc: 0.3453 - val_loss: 10.1444 - val_acc: 0.3551

Epoch 00001: val_loss improved from inf to 10.14438, saving model to weights.01-10.14.hdf5
Epoch 2/10
530/530 [==============================] - 68s 129ms/step - loss: 11.1393 - acc: 0.3075 - val_loss: 11.9003 - val_acc: 0.2617

Epoch 00002: val_loss did not improve from 10.14438
Epoch 3/10
530/530 [==============================] - 67s 127ms/step - loss: 12.9553 - acc: 0.1962 - val_loss: 12.5028 - val_acc: 0.2243

Epoch 00003: val_loss did not improve from 10.14438
Epoch 4/10
530/530 [==============================] - 68s 128ms/step - loss: 12.5600 - acc: 0.2208 - val_loss: 12.5028 - val_acc: 0.2243

Epoch 00004: val_loss did not improve from 10.14438
Epoch 5/10
530/530 [==============================] - 71s 135ms/step - loss: 12.5600 - acc: 0.2208 - val_loss: 12.5028 - val_acc: 0.2243

Epoch 00005: val_l